In [1]:
import numpy as np
from DSSStartup import DSSStartup
from setInfo import *
from getInfo import *
import matplotlib.pyplot as plt
from math import tan,acos

# Global variable initialization and error checking - ok

In [2]:
Sbase=1
LoadScalingFactor = 7
GenerationScalingFactor = 5 
SlackBusVoltage = 1.05 
NoiseMultiplyer= 0

#Set simulation analysis period - the simulation is from StartTime to EndTime
StartTime = 40000 
EndTime = 40500
EndTime += 1 # creating a list, last element does not count, so we increase EndTime by 1
#Set hack parameters
TimeStepOfHack = 50
PercentHacked = np.array([0.5, 0, 1.0, 0, 0, 0.5, 0, 0.5, 0, 0, 0.5, 0.5, 0.5, 0,
                        0, 0, 1.0, 0, 0, 0.5, 0, 0.5, 0, 0.5, 0.5, 0, 0.5, 0, 0.5, 1, 0])

#Set initial VBP parameters for uncompromised inverters
VQ_start = 1.01
VQ_end = 1.03
VP_start = 1.03
VP_end = 1.05

#Set adaptive controller gain values (the higher the gain, the faster the response time)
kq = 1
kp = 1

#Set delays for each node
                        #1  2  3   4   5  6   7   8      9    10   11   12   13
Delay_VoltageSampling = np.array([0, 0, 10, 0, 0,  10,  10,  50,  10,  10,  10,  10,  10]) 
Delay_VBPCurveShift =   np.array([0, 0, 120, 0, 0, 120, 120, 120, 120, 120, 120, 120, 120])

#Set observer voltage threshold
ThreshHold_vqvp = 0.25
power_factor=0.9
pf_converted=tan(acos(power_factor))
Number_of_Inverters = 13 #even feeder is 34Bus, we only have 13 inverters

#The following variable allows to run the simulation without any inverter
SimulateInverterHack=1

#Error checking of the global variable -- TODO: add error handling here!
if EndTime < StartTime or EndTime < 0 or StartTime < 0:
    print('Setup Simulation Times Appropriately.')
if NoiseMultiplyer < 0:
    print('Setup Noise Multiplyer Correctly.')

# Initialization of OpenDSS - ok

In [3]:
DSSStart = DSSStartup()
DSSText =DSSStart['dsstext']
DSSSolution = DSSStart['dsssolution']
DSSCircuit = DSSStart['dsscircuit']
DSSObj = DSSStart['dssobj']
DSSMon = DSSCircuit.Monitors
DSSText.command = 'Compile C:\\feeders\\feeder34_B_NR\\feeder34_B_NR.dss'
    
DSSSolution.Solve();
if not DSSSolution.Converged:
    print('Initial Solution Not Converged. Check Model for Convergence')
else:
    print('Initial Model Converged. Proceeding to Next Step.')
    #Doing this solve command is required for GridPV, that is why the monitors
    #go under a reset process
    DSSMon.ResetAll
    setSolutionParams(DSSObj,'daily',1,1,'off',1000000,30000)
    #Easy process to get all names and count of loads, a trick to avoid
    #some more lines of code
    TotalLoads=DSSCircuit.Loads.Count
    AllLoadNames=DSSCircuit.Loads.AllNames
    print('OpenDSS Model Compliation Done.')

Initial Model Converged. Proceeding to Next Step.
OpenDSS Model Compliation Done.


# Load data from file

In [4]:
#Retrieving the data from the load profile
TimeResolutionOfData=10 #resolution in minute
#Get the data from the Testpvnum folder
#Provide Your Directory - move testpvnum10 from github to drive C: 
FileDirectoryBase ='C:\\feeders\\testpvnum10\\';
QSTS_Time = list(range(1441)) #This can be changed based on the available data - for example, 1440 timesteps
QSTS_Data = np.zeros((len(QSTS_Time) ,4,TotalLoads)) #4 columns as there are four columns of data available in the .mat file

for node in range(TotalLoads):
    #This is created manually according to the naming of the folder
    FileDirectoryExtension = 'node_' + str(node+1) + '_pv_' +str(TimeResolutionOfData) + '_minute.csv'
    #The total file directory
    FileName = FileDirectoryBase + FileDirectoryExtension
    #Load the file
    MatFile = np.genfromtxt(FileName, delimiter=',')    
    QSTS_Data[:,:,node] = MatFile #Putting the loads to appropriate nodes according to the loadlist
    
Generation = QSTS_Data[:,1,:]*GenerationScalingFactor #solar generation
Load = QSTS_Data[:,3,:]*LoadScalingFactor #load demand
Generation = np.squeeze(Generation)/Sbase  #To convert to per unit, it should not be multiplied by 100
Load = np.squeeze(Load)/Sbase
print('Reading Data for Pecan Street is done.')

Reading Data for Pecan Street is done.


# Interpolate to change data from minutes to seconds - ok

In [26]:
from scipy.interpolate import interp1d

print('Starting Interpolation...')

#interpolation for the whole period...
Time = list(range(StartTime,EndTime))
TotalTimeSteps = len(Time)
LoadSeconds = GenerationSeconds = np.empty([3600*24, TotalLoads])
# Interpolate to get minutes to seconds
for node in range(TotalLoads): # i is node
    t_seconds = np.linspace(1,len(Load[:,node]), int(3600*24/1))
    f = interp1d(range(len(Load[:,node])), Load[:,node], kind='cubic', fill_value="extrapolate")
    LoadSeconds[:,node] = f(t_seconds) #spline method in matlab equal to Cubic Spline -> cubic
    
    f = interp1d(range(len(Generation[:,node])), Generation[:,node], kind='cubic', fill_value="extrapolate")
    GenerationSeconds[:,node]= f(t_seconds)

# Initialization
# then we take out only the window we want...
LoadSeconds = LoadSeconds[StartTime:EndTime,:]
GenerationSeconds = GenerationSeconds[StartTime:EndTime,:]
Load = LoadSeconds
Generation = GenerationSeconds

#Create noise vector
Noise = np.empty([TotalTimeSteps, TotalLoads])
for node in range(TotalLoads):
    Noise[:,node] = np.random.randn(TotalTimeSteps) 

#Add noise to loads
for node in range(TotalLoads):
    Load[:,node] = Load[:,node] + NoiseMultiplyer*Noise[:,node]

if NoiseMultiplyer > 0:
    print('Load Interpolation has been done. Noise was added to the load profile.') 
else:
    print('Load Interpolation has been done. No Noise was added to the load profile.') 
    
MaxGenerationPossible = np.max(Generation, axis = 0)

Starting Interpolation...
Load Interpolation has been done. No Noise was added to the load profile.


# Inverter Object

In [ ]:
class inverter(object):
    __init__(self, name, Delay_VoltageSampling=10, Delay_VBPCurveShift=120, InverterLPF=1, LowPassFilterFrequency=0.1,
            HighPassFilterFrequency=1, Gain_Energy=1e5,ThreshHold_vqvp=0.25, PercentHacked=0, InverterRateOfChangeActivate=0,
            InverterRateOfChangeLimit = 100):
        
        #value inside
        self.TimeStep = 1
        
        #unchange value
        self.name = name
        self.Delay_VoltageSampling = Delay_VoltageSampling
        self.Delay_VBPCurveShift = Delay_VBPCurveShift
        self.InverterLPF = InverterLPF
        self.LowPassFilterFrequency = LowPassFilterFrequency
        self.HighPassFilterFrequency = HighPassFilterFrequency
        self.Gain_Energy = Gain_Energy
        self.ThreshHold_vqvp = ThreshHold_vqvp
        self.PercentHacked = PercentHacked
        self.InverterRateOfChangeActivate = InverterRateOfChangeActivate
        self.InverterRateOfChangeLimit = InverterRateOfChangeLimit

        #stored variables
        self.V_m1 = 0
        
        self.FilterVoltage_m1 = 0
        self.FilterVoltageCalc_m1 = 0
        self.InvReal_m1 = 0
        self.InvReact_m1 = 0
        self.VBP = np.array([0., 0., 0., 0.])
        self.FilteredOutput_vqvp_m1 = 0
        self.IntermediateOutput_vqvp_m1 = 0
        self.Epsilon_vqvp_m1 = 0
        
    def inverter_VVVW_model(self, currentVoltage, SolarGeneration_vqvp, ksim):
        # we return new value of InvReact and Real, but we made it as our observation in the past also for the next iter
        self.InvReact_m1, self.InvReal_m1, self.FilterVoltage_m1, self.FilterVoltageCalc_m1, _, _ = inverter_VoltVarVoltWatt_model(
                     self.FilterVoltage_m1, SolarGeneration_vqvp, 
                     abs(currentVoltage), abs(self.V_m1), 
                     self.VBP, self.TimeStep, self.InverterLPF, 
                     self.Sbar, self.InvReal_m1, 
                     self.InvReact_m1, self.InverterRateOfChangeLimit, 
                     self.InverterRateOfChangeActivate, ksim, self.Delay_VoltageSampling)
        
        #we still need V_m1 for observer so we dont update V_m1 here...
        
        
        return self.InvReal_m1, self.InvReact_m1, self.FilterVoltage_m1, self.FilterVoltageCalc_m1
    
    
    def inverter_observer(self, currentVoltage, ksim):
        voltage_observer(InverterArray(knode),V_vqvp(knode,ksim), ...
                V_vqvp(knode,ksim-1), IntermediateOutput_vqvp(ksim-1,knode), ...
                Epsilon_vqvp(ksim-1,knode), FilteredOutput_vqvp(ksim-1,knode))
        

# Environment Object -
this is actually a big simulation object manage and running the simulation, calling and using OpenDSS